<h1><center>Laboratorio 9: Benchmark Estadístico con Reddit 🧮</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos</strong></center>

### Cuerpo Docente:

- Profesor: Pablo Badilla
- Auxiliar: Ignacio Meza D.
- Ayudante: Constanza Peña

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1: Kevin Pinochet Hernández
- Nombre de alumno 2:


### **Link de repositorio de GitHub:** `https://github.com/Keviinplz/MDS7202-1`

# Temas a tratar

- Optimización de Código en Python.
- Utilización de librerías para medir el tiempo de ejecución de funciones.
- Métodos para optimizar el rendimiento de las funciones.

# Reglas:

- Fecha de entrega: 3/12/2021
- **Grupos de 2 personas**
- **Ausentes** deberán realizar la actividad solos. 
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias. 
- Pueden usar cualquer matrial del curso que estimen conveniente.

# Objetivos principales del laboratorio

- Obtener datos desde Reddit a partir de la librería `praw` y visualizar cuales post son más probables que sean puntuados positivamente.
- Aplicar un atajo estadístico para obtener la mean posterior de datos.
- Optimizar a través de librerías funciones.

#Importamos librerias utiles 😸

In [1]:
%%capture
!pip install praw
!pip install line_profiler
!pip install memory_profiler
!pip install numba

import sys
import praw
import time
import numpy as np
import pandas as pd
from numba import jit
import plotly.express as px
from functools import lru_cache
from IPython.core.display import Image

# 1. Recomendando Posts de Subreddit

<p align="center">
  <img src="https://styles.redditmedia.com/t5_2rer8/styles/bannerBackgroundImage_xzrb6my122181.jpg">
</p>

Unos intrépidos alumnos del curso, quienes frecuentemente participan en subreddits y foros de reviews, se preguntan lo siguiente: ¿Podremos confiar que un post es bueno, si este tiene solamente 1 o 3 votos positivos?. los compañeros, creen que esto claramente no representa una opinión general, ya que estamos mucho menos seguros acerca de la verdadera proporción de votos a favor de los comentarios con pocos datos. ¿Pero cómo podemos obtener una representación más creíble para este problema?.

Lo señalado forma parte de un problema estadísticos, donde a través del cálculo de la posterior se puede conocer que tan probable es que un post sea bueno. Para efectos de este laboratorio, no se exige un conocimiento previo para resolver este problema, simplemente se deberá aplicar las ecuaciones presentadas más adelante (De igual forma si quedan interesados sobre el tema se les invita a tomar el ramo [CC6104](https://github.com/dccuchile/CC6104)).

## 1.1 Obtención de Subrredits y Análisis [1 Punto]

Para estudiar que tan probable es que un post sea positivo se comenzará cargando datos reales del subreddit chile desde reddit (si usted desea puede cambiar el subreddit a uno de su gusto). Para esto le proponemos la utilización de la función que aparece mas abajo, la que presenta un usuario ya creado por el equipo docente. Dese un tiempo para entender que hace cada parte de la función, visualizando que se obtiene de estas.

Revisada la función, utilice un **perfilador** para monitorear el tiempo y memoria que les toma a cada liena de código para ser ejecutada. Señale cuales son los procesos que mas tiempo consumen en la ejecución del Código, comentando si es posible mejorar el desempeño de la función.

**TO-DO:**
- [ ] Estudiar la función propuesta por el equipo docente.
- [ ] Estudiar los tiempos de ejecución del código a través de un perfilador.
- [ ] Estudiar la memoria ocupada por el código a través de un perfilador.
- [ ] Comente los resultados. ¿Qué lineas ocupan más memoria o más/menos tiempo?¿Por qué?

In [2]:
import praw
import numpy as np

def praw_reddit(nombre_subreddit="chile", n_hot=1000):
    reddit = praw.Reddit(
        client_id="-w2hyFINxZ8T3g",
        client_secret="zGPCI4s3g6Ic6AsRi7vIpP0NoxbFdw",
        password="ClasesMDS7202",
        user_agent="Clases",
        username="DocenciaDataScience",
        check_for_async=False,
    )
    subreddit = reddit.subreddit(nombre_subreddit)

    votes, post, url = {}, {}, {}
    top_submissions = list(subreddit.hot(limit=n_hot))
    for it, top_n in enumerate(range(50, len(top_submissions), 50)):
        top_n_submissions = top_submissions[:top_n]
        upvotes, downvotes, url[it], post[it] = [], [], [], []

        for submission in top_n_submissions:
            try:
                ratio = submission.upvote_ratio
                ups = int(
                    round((ratio * submission.score) / (2 * ratio - 1))
                    if ratio != 0.5
                    else round(submission.score / 2)
                )
                upvotes.append(ups)
                downvotes.append(ups - submission.score)
                post[it].append(submission.title)
                url[it].append(submission.url)
            except Exception as e:
                continue
        votes[it] = np.array([upvotes, downvotes]).T
    return votes, post, url




In [3]:
votes, post, url = praw_reddit()

In [4]:
%%file bench_reddit.py
import praw
import numpy as np

def praw_reddit(nombre_subreddit="chile", n_hot=1000):
    reddit = praw.Reddit(
        client_id="-w2hyFINxZ8T3g",
        client_secret="zGPCI4s3g6Ic6AsRi7vIpP0NoxbFdw",
        password="ClasesMDS7202",
        user_agent="Clases",
        username="DocenciaDataScience",
        check_for_async=False,
    )
    subreddit = reddit.subreddit(nombre_subreddit)

    votes, post, url = {}, {}, {}
    top_submissions = list(subreddit.hot(limit=n_hot))
    for it, top_n in enumerate(range(50, len(top_submissions), 50)):
        top_n_submissions = top_submissions[:top_n]
        upvotes, downvotes, url[it], post[it] = [], [], [], []

        for submission in top_n_submissions:
            try:
                ratio = submission.upvote_ratio
                ups = int(
                    round((ratio * submission.score) / (2 * ratio - 1))
                    if ratio != 0.5
                    else round(submission.score / 2)
                )
                upvotes.append(ups)
                downvotes.append(ups - submission.score)
                post[it].append(submission.title)
                url[it].append(submission.url)
            except Exception as e:
                continue
        votes[it] = np.array([upvotes, downvotes]).T
    return votes, post, url

Overwriting bench_reddit.py


In [5]:
###### Código Aquí ######
%load_ext line_profiler
%lprun -f praw_reddit praw_reddit()

Timer unit: 1e-07 s

Total time: 15.3569 s
File: C:\Users\Kevii\AppData\Local\Temp/ipykernel_22064/3471271660.py
Function: praw_reddit at line 4

Line #      Hits         Time  Per Hit   % Time  Line Contents
     4                                           def praw_reddit(nombre_subreddit="chile", n_hot=1000):
     5         2       7880.0   3940.0      0.0      reddit = praw.Reddit(
     6         1         13.0     13.0      0.0          client_id="-w2hyFINxZ8T3g",
     7         1          9.0      9.0      0.0          client_secret="zGPCI4s3g6Ic6AsRi7vIpP0NoxbFdw",
     8         1          8.0      8.0      0.0          password="ClasesMDS7202",
     9         1          9.0      9.0      0.0          user_agent="Clases",
    10         1          8.0      8.0      0.0          username="DocenciaDataScience",
    11         1         10.0     10.0      0.0          check_for_async=False,
    12                                               )
    13         1        369.0    369.

In [6]:
%load_ext memory_profiler
from bench_reddit import praw_reddit

%mprun -f praw_reddit praw_reddit()

Filename: c:\Users\Kevii\Documents\Universidad\mds7202\repo\labs\lab9\bench_reddit.py

Line #    Mem usage    Increment  Occurences   Line Contents
     4    133.9 MiB    133.9 MiB           1   def praw_reddit(nombre_subreddit="chile", n_hot=1000):
     5    133.9 MiB      0.0 MiB           2       reddit = praw.Reddit(
     6    133.9 MiB      0.0 MiB           1           client_id="-w2hyFINxZ8T3g",
     7    133.9 MiB      0.0 MiB           1           client_secret="zGPCI4s3g6Ic6AsRi7vIpP0NoxbFdw",
     8    133.9 MiB      0.0 MiB           1           password="ClasesMDS7202",
     9    133.9 MiB      0.0 MiB           1           user_agent="Clases",
    10    133.9 MiB      0.0 MiB           1           username="DocenciaDataScience",
    11    133.9 MiB      0.0 MiB           1           check_for_async=False,
    12                                             )
    13    133.9 MiB      0.0 MiB           1       subreddit = reddit.subreddit(nombre_subreddit)
    14            

In [7]:
votes

{0: array([[  93,   19],
        [  36,   13],
        [ 283,    3],
        [ 482,   42],
        [ 103,    5],
        [ 142,    7],
        [ 193,    8],
        [ 161,   20],
        [ 141,    7],
        [  33,    0],
        [ 152,    3],
        [  30,    0],
        [  40,    1],
        [ 119,    9],
        [ 251,   13],
        [  25,    4],
        [  32,    1],
        [  45,   20],
        [  62,   11],
        [  27,    2],
        [  16,    2],
        [  29,    2],
        [  46,    7],
        [  41,    4],
        [  70,   15],
        [  45,    9],
        [  13,    4],
        [  51,    2],
        [  42,   12],
        [  95,   18],
        [ 461,   29],
        [  51,   20],
        [  10,    4],
        [   5,    0],
        [ 260,   20],
        [ 308,    6],
        [ 841,   17],
        [ 179,   22],
        [ 253,   22],
        [   9,    3],
        [  21,    5],
        [ 169,    7],
        [  77,    7],
        [1067,   22],
        [ 136,   17],
       

In [8]:
post

{0: ['Discusión random semanal',
  'FYI: Los memes políticos están temporalmente no permitidos en el sub.',
  'Araucaria y atardecer en el Parque Nacional Conguillío Región de la Araucanía',
  '"Niño Poeta" se define en segunda vuelta: "Partí siendo de derecha, pero con el tiempo la vida te va izquierdizando"',
  'Perfil del chileno que usa Reddit',
  'de estos quién lo haría mejor como presidente?',
  'Mi ex me robó a mi perro',
  'Naranjo y carta oficialista a la ONU por Bachelet: “El circo de los Tachuelas parece que salió en gira internacional”',
  'Allamand por apoyo de Bachelet a Boric en segunda vuelta: “Desde Cancillería no vemos inconvenientes en que la expresidenta, como cualquier chilena, exprese su preferencia electoral”',
  'Secta en Papudo',
  'AKA Yisus',
  'Femicida con salida dominical huyó de la cárcel para secuestrar y matar a su actual pareja',
  '6 aviones perdonen la calidad',
  'Ricardo Lagos: "La extrema izquierda era la mejor propagandista para llevar votos al 

In [9]:
url

{0: ['https://www.reddit.com/r/chile/comments/rfbhun/discusión_random_semanal/',
  'https://www.reddit.com/r/chile/comments/rhaf3i/fyi_los_memes_políticos_están_temporalmente_no/',
  'https://i.imgur.com/cyFFmuY.jpg',
  'https://oasisfm.cl/nino-poeta-entrevista-boric-kast-15-12-2021',
  'https://www.reddit.com/r/chile/comments/rh6j4n/perfil_del_chileno_que_usa_reddit/',
  'https://i.redd.it/o9re9osebq581.jpg',
  'https://www.reddit.com/r/chile/comments/rgzpam/mi_ex_me_robó_a_mi_perro/',
  'https://www.cnnchile.com/elecciones2021/narajo-carta-bachelet-onu-circo-tachuelas_20211215/',
  'https://www.latercera.com/politica/noticia/allamand-por-apoyo-de-michelle-bachelet-a-boric-en-segunda-vuelta-desde-la-cancilleria-no-vemos-inconvenientes-en-que-la-expresidenta-como-cualquier-chilena-exprese-su-preferencia-electoral/QRIDZNHIKVEDHN7EDIS444NP2M/',
  'https://www.reddit.com/r/chile/comments/rhbosw/secta_en_papudo/',
  'https://imgur.com/wux2d96',
  'https://cooperativa.cl/noticias/pais/polic

**Respuesta:**

`La función analiza los post actuales de la subred /chile, donde toma un top 50 y calcula los votos positivos y negativos, el titulo del post, y la url asociada`
`Realiza esta operación la cantidad de veces definidas por la función, siempre de 50 en 50, ordenados por las más rankeadas, a la menos rankeadas`
`Finalmente retorna los votos, el titulo, y la url`

`Se puede apreciar, que lo que demora más tiempo y toma más memoria, es crear la lista de posts encontrados (por defecto, 1000)`
`A su vez, toma tiempo calcular la cantidad de votos positivos, una fracción muy menor comparado lo que toma crear la lista, pero`
`suficientemente grande como para tomar un porcentaje`

`Crear la lista toma tiempo porque se usan las listas nativas de python, al ser listas que pueden almacenar cualquier tipo de objeto`
`entonces provoca que se tenga que solicitar mayor memoria de la esperaba para el tipo de objeto que se quiere guardar`

`A su vez, calcular los votos positivos toma tiempo de computo debido a que existe una comparación que gasta tiempo en CPU.`

## 1.2 LRU y Análisis de Tiempo con Cache [1 Punto]

Respondan las siguientes preguntas:

1.	¿Qué es la memoria cache y a que se refiere las siglas LRU?
2.	¿Cuáles son los costos que tiene la aplicación de técnicas de Caching?
3. ¿Cuál es la consecuencía de ocupar caching en la función anterior?.

Respondidas las preguntas, se le solicita que aplique alguna técnica de caching para mejorar el desempeño de la función `praw_reddit`. Para esto compare solo el tiempo de ejecución del algoritmo con y sin caching, señalando el tiempo total de ejecución y el tiempo promedio que le toma ejecutar cada loop a la función. Con lo anterior, ¿es posible visualizar mejoras en este caso?.


**TO-DO:**
- [ ] Responder las preguntas.
- [ ] Mejorar el código con cache.
- [ ] Explicar las consecuencias de usar lru_cache sobre praw_reddit

**Respuestas Teóricas:**

`Es un tipo de memoria en la que se va almacenando ejecuciones calculadas de alguna operación `
`para luego ser usada posteriormente sin tener que volver a calcular.`

`Un ejemplo de memoria caché es LRU o Last Recent Used, donde utilizando la tecnica de memoización, almacena `
`valores precomputados que son requeridos constantemente, por ejemplo, la función fibbonacci.`

`El problema de usar estos métodos es obviamente el uso de memoria, si la memoria está alojada en otra parte que no fuera RAM `
`entonces el rendimiento de la función decaería drásticamente.`

`Otro problema es cuando la función no necesariamente retorna lo mismo al aplicarse el mismo predicado, en lenguajes como Haskell esto no ocurre `
`pero en Python si puede ocurrir, en ese caso usar caché podría generar que la función siempre retorne lo mismo, cuando no es lo esperado. `
`Por ejemplo`

```py
from datetime import datetime
from functools import lru_cache

@lru_cache
def now() -> datetime:
    return datetime.now()
```

`La primera ejecución de now() retornaría la fecha actual, pero la segunda ejecución no, dado que hemos almacenado en caché el valor de la función`

In [10]:
from time import sleep 
from datetime import datetime
from functools import lru_cache

@lru_cache
def now() -> datetime:
    return datetime.now()

print(f"Fecha actual: {now().strftime('%d/%m/%Y %H:%M:%S')}")
sleep(2)
print(f"Fecha actual: {now().strftime('%d/%m/%Y %H:%M:%S')}")

Fecha actual: 15/12/2021 20:56:21
Fecha actual: 15/12/2021 20:56:21


`Usar caching en la función anterior (praw_reddit) provocaría que ya no se pidan los post en tiempo real, dado que usará los valores de vote, post y url `
`que logró capturar en la primera ejecución, sin embargo, mejoraría su rendimiento, puesto que `

In [11]:
###### Código Aquí ######
# Pre cache

import praw
import numpy as np

def praw_reddit(nombre_subreddit="chile", n_hot=1000):
    reddit = praw.Reddit(
        client_id="-w2hyFINxZ8T3g",
        client_secret="zGPCI4s3g6Ic6AsRi7vIpP0NoxbFdw",
        password="ClasesMDS7202",
        user_agent="Clases",
        username="DocenciaDataScience",
        check_for_async=False,
    )
    subreddit = reddit.subreddit(nombre_subreddit)

    votes, post, url = {}, {}, {}
    top_submissions = list(subreddit.hot(limit=n_hot))
    for it, top_n in enumerate(range(50, len(top_submissions), 50)):
        top_n_submissions = top_submissions[:top_n]
        upvotes, downvotes, url[it], post[it] = [], [], [], []

        for submission in top_n_submissions:
            try:
                ratio = submission.upvote_ratio
                ups = int(
                    round((ratio * submission.score) / (2 * ratio - 1))
                    if ratio != 0.5
                    else round(submission.score / 2)
                )
                upvotes.append(ups)
                downvotes.append(ups - submission.score)
                post[it].append(submission.title)
                url[it].append(submission.url)
            except Exception as e:
                continue
        votes[it] = np.array([upvotes, downvotes]).T
    return votes, post, url

%load_ext line_profiler
%lprun -f praw_reddit praw_reddit()

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


Timer unit: 1e-07 s

Total time: 16.2923 s
File: C:\Users\Kevii\AppData\Local\Temp/ipykernel_22064/929997915.py
Function: praw_reddit at line 7

Line #      Hits         Time  Per Hit   % Time  Line Contents
     7                                           def praw_reddit(nombre_subreddit="chile", n_hot=1000):
     8         2       5659.0   2829.5      0.0      reddit = praw.Reddit(
     9         1          9.0      9.0      0.0          client_id="-w2hyFINxZ8T3g",
    10         1          5.0      5.0      0.0          client_secret="zGPCI4s3g6Ic6AsRi7vIpP0NoxbFdw",
    11         1          6.0      6.0      0.0          password="ClasesMDS7202",
    12         1          6.0      6.0      0.0          user_agent="Clases",
    13         1          5.0      5.0      0.0          username="DocenciaDataScience",
    14         1          5.0      5.0      0.0          check_for_async=False,
    15                                               )
    16         1        492.0    492.0

In [12]:
%load_ext memory_profiler
from bench_reddit import praw_reddit

%mprun -f praw_reddit praw_reddit()

The memory_profiler extension is already loaded. To reload it, use:
  %reload_ext memory_profiler



Filename: c:\Users\Kevii\Documents\Universidad\mds7202\repo\labs\lab9\bench_reddit.py

Line #    Mem usage    Increment  Occurences   Line Contents
     4    148.4 MiB    148.4 MiB           1   def praw_reddit(nombre_subreddit="chile", n_hot=1000):
     5    148.4 MiB      0.0 MiB           2       reddit = praw.Reddit(
     6    148.4 MiB      0.0 MiB           1           client_id="-w2hyFINxZ8T3g",
     7    148.4 MiB      0.0 MiB           1           client_secret="zGPCI4s3g6Ic6AsRi7vIpP0NoxbFdw",
     8    148.4 MiB      0.0 MiB           1           password="ClasesMDS7202",
     9    148.4 MiB      0.0 MiB           1           user_agent="Clases",
    10    148.4 MiB      0.0 MiB           1           username="DocenciaDataScience",
    11    148.4 MiB      0.0 MiB           1           check_for_async=False,
    12                                             )
    13    148.4 MiB      0.0 MiB           1       subreddit = reddit.subreddit(nombre_subreddit)
    14            

In [13]:
from functools import lru_cache
import praw
import numpy as np

@lru_cache
def praw_reddit_cache(nombre_subreddit="chile", n_hot=1000):
    reddit = praw.Reddit(
        client_id="-w2hyFINxZ8T3g",
        client_secret="zGPCI4s3g6Ic6AsRi7vIpP0NoxbFdw",
        password="ClasesMDS7202",
        user_agent="Clases",
        username="DocenciaDataScience",
        check_for_async=False,
    )
    subreddit = reddit.subreddit(nombre_subreddit)

    votes, post, url = {}, {}, {}
    top_submissions = list(subreddit.hot(limit=n_hot))
    for it, top_n in enumerate(range(50, len(top_submissions), 50)):
        top_n_submissions = top_submissions[:top_n]
        upvotes, downvotes, url[it], post[it] = [], [], [], []

        for submission in top_n_submissions:
            try:
                ratio = submission.upvote_ratio
                ups = int(
                    round((ratio * submission.score) / (2 * ratio - 1))
                    if ratio != 0.5
                    else round(submission.score / 2)
                )
                upvotes.append(ups)
                downvotes.append(ups - submission.score)
                post[it].append(submission.title)
                url[it].append(submission.url)
            except Exception as e:
                continue
        votes[it] = np.array([upvotes, downvotes]).T
    return votes, post, url

%load_ext line_profiler
%lprun -f praw_reddit_cache praw_reddit_cache()

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


Timer unit: 1e-07 s

In [14]:
%%file bench_reddit_cache.py

from functools import lru_cache

import praw
import numpy as np

@lru_cache
def praw_reddit_cache(nombre_subreddit="chile", n_hot=1000):
    reddit = praw.Reddit(
        client_id="-w2hyFINxZ8T3g",
        client_secret="zGPCI4s3g6Ic6AsRi7vIpP0NoxbFdw",
        password="ClasesMDS7202",
        user_agent="Clases",
        username="DocenciaDataScience",
        check_for_async=False,
    )
    subreddit = reddit.subreddit(nombre_subreddit)

    votes, post, url = {}, {}, {}
    top_submissions = list(subreddit.hot(limit=n_hot))
    for it, top_n in enumerate(range(50, len(top_submissions), 50)):
        top_n_submissions = top_submissions[:top_n]
        upvotes, downvotes, url[it], post[it] = [], [], [], []

        for submission in top_n_submissions:
            try:
                ratio = submission.upvote_ratio
                ups = int(
                    round((ratio * submission.score) / (2 * ratio - 1))
                    if ratio != 0.5
                    else round(submission.score / 2)
                )
                upvotes.append(ups)
                downvotes.append(ups - submission.score)
                post[it].append(submission.title)
                url[it].append(submission.url)
            except Exception as e:
                continue
        votes[it] = np.array([upvotes, downvotes]).T
    return votes, post, url



Overwriting bench_reddit_cache.py


In [84]:
from bench_reddit_cache import praw_reddit_cache

%mprun -f bench_reddit_cache praw_reddit_cache()

# No logré que se mostrara esto :l sacrifico mi punto :( 

UsageError: Could not find function 'bench_reddit_cache'.
NameError: name 'bench_reddit_cache' is not defined


**Respuesta de lru_cache sobre praw_reddit**

`Escriba su respuesta aquí`

## 1.3 Obtención de Mean Posterior y Standard Error [1 Punto]

Una forma de obtener la mean posterior y el Standard Error de los datos de reddit es aplicando un atajo de inferencia estadística. Este atajo se define de la siguiente forma:

Sea **u** los votos positivos y **d** los votos negativos del subreddit, tendremos que:

$$a = 1+u$$

$$b = 1+d$$

$$\sigma= 1.65\sqrt(\dfrac{ab}{(a + b)^2(a + b + 1)})$$

$$\mu = \dfrac{a}{a+b}$$

Donde $\mu$ es la mean posterior y $\sigma$ el standard error.

Con lo anterior, genere dos funciones que tengan como salida $\mu$ y $\sigma$ de acuerdo a las ecuaciones señaladas. La primera función, deberá ser construida sin el uso de numpy, aplicando for y aplicando comandos nativos de Python. Por otro lado, deberá generar una segunda función con el uso exclusivo de numpy. **OJO** que las funciones deben tener como entrada solo un elemento del diccionario votes (por ejemplo `votes[1]`), por lo que estas no deben tener como entrada el conjunto completo de datos.

In [72]:
from typing import Tuple

import math
import numpy as np

def p_mean_and_std(u, d):
    mus, sigmas = [], []
    for vote in votes:
        a = vote[0] + 1
        b = vote[1] + 1

        sigma = 1.65 * (1 / (a+b)) * math.sqrt(a*b/((a + b + 1)))
        mu = a / (a + b)
        mus.append(mu)
        sigmas.append(sigma)

    return mus, sigmas

def p_mean_and_std_optimized(votes):
    votes = votes + 1
    a = votes[:, 0]
    b = votes[:, 1]

    sigma = 1.65 * (1 / (a+b) * np.sqrt(a*b/(a + b + 1)))
    mu = a / (a + b)

    return mu, sigma

## 1.4 Comparación de Rendimiento [2 Puntos]

Creadas las funciones de la sección 1.3, de forma iterativa compare los diferentes batches de datos que contiene el diccionario `votes`. Para esto genere un gráfico utilizando plotly, donde se pueda observar las diferencias de tiempo que toma ejecutar las diferentes cantidades de datos. ¿es posible observar una diferencia? ¿a qué se deberá esto?.

Aplique el compilador **Numba** sobre las funciones 1.3 y compare el tiempo de ejecución con los obtenidos sin el compilador. 

In [77]:
###### Código Aquí ######
import time

import pandas as pd
import plotly.express as px

native: dict = {}
optimized: dict = {}

for key, batch in votes.items():
    t0 = time.perf_counter_ns()
    p_mean_and_std(batch)
    t1 = time.perf_counter_ns()
    native[key] = (t1 - t0) / 1000

    t0 = time.perf_counter_ns()
    p_mean_and_std_optimized(batch)
    t1 = time.perf_counter_ns()
    optimized[key] = (t1 - t0) / 1000

df = pd.DataFrame.from_dict(
    { "x": list(native.keys()), "Python Nativo": list(native.values()), "Python Numpy": list(optimized.values()) }
)

fig = px.line(df, x='x', y=["Python Nativo", "Python Numpy"], title="Comparación de tiempos de ejecución en microsegundos")
fig.show()

In [74]:
import math
import numpy as np

@jit
def p_mean_and_std(votes):
    mus, sigmas = [], []
    for vote in votes:
        a = vote[0] + 1
        b = vote[1] + 1

        sigma = 1.65 * (1 / (a+b)) * math.sqrt(a*b/((a + b + 1)))
        mu = a / (a + b)
        mus.append(mu)
        sigmas.append(sigma)

    return mus, sigmas

@jit(nopython=True)
def p_mean_and_std_optimized(votes):
    votes = votes + 1
    a = votes[:, 0]
    b = votes[:, 1]

    sigma = 1.65 * (1 / (a+b) * np.sqrt(a*b/(a + b + 1)))
    mu = a / (a + b)

    return mu, sigma

Ahora, grafiquen los tiempos tomados y comente los desempeños.

**Nota:** Ejecuten las funciones compiladas con algún ejemplo antes de graficar. De lo contrario, les indicará el tiempo de compilación, cosa que no queremos medir

In [78]:
###### Código Aquí ######
import time

import pandas as pd
import plotly.express as px

native: dict = {}
optimized: dict = {}

p_mean_and_std(votes[0])
p_mean_and_std_optimized(votes[0])

for key, batch in votes.items():
    t0 = time.perf_counter_ns()
    p_mean_and_std(batch)
    t1 = time.perf_counter_ns()
    native[key] = (t1 - t0) / 1000

    t0 = time.perf_counter_ns()
    p_mean_and_std_optimized(batch)
    t1 = time.perf_counter_ns()
    optimized[key] = (t1 - t0) / 1000

df = pd.DataFrame.from_dict(
    { "x": list(native.keys()), "Python Nativo (Numba)": list(native.values()), "Python Numpy (Numba)": list(optimized.values()) }
)

fig = px.line(df, x='x', y=["Python Nativo (Numba)", "Python Numpy (Numba)"], title="Comparación de tiempos de ejecución en microsegundos")
fig.show()

In [81]:
from typing import Tuple

import math
import numpy as np


def p_mean_and_std_optimized(votes):
    votes = votes + 1
    a = votes[:, 0]
    b = votes[:, 1]

    sigma = 1.65 * (1 / (a+b) * np.sqrt(a*b/(a + b + 1)))
    mu = a / (a + b)

    return mu, sigma

## 1.5 Plot de Resultados [1 Punto]

Llego la hora de visualizar los resultados obtenidos, para esto solo ejecute las siguientes celdas y observe lo que sucede :3. ¿Qué logra observar de los resultados?, ¿La solución resulta trivial?

En esta sección esperamos que solo comenten con lo que logran visualizar de los dataframes.

In [83]:
###### Código Aquí ######
votos = votes[len(votes) - 1]
posteo = post[len(post) - 1]
print("lower bounds aproximados:")
posterior_mean, std_err = p_mean_and_std_optimized(votos)
lb = posterior_mean - std_err
print("\nTop 20 post ordenador por el limite inferior:\n")
order = np.argsort(-lb)
vote_post = {
    'Votos (+)': votos[order[:20], 0],
    'Votos (-)': votos[order[:20], 1],
    'Post': np.array(posteo)[order[:20]],
    'url': np.array(url[len(url) - 1])[order[:20]]
}
df = pd.DataFrame(data=vote_post)
ordered_post = df.Post

r_order = order[::-1][-20:]
data_dic = {
    'mean': posterior_mean[r_order],
    'std_err': std_err[r_order],
    'post': ordered_post[::-1]
}
df = pd.DataFrame(data=data_dic)
fig = px.scatter(df, x="mean", y="post", error_x="std_err")
fig.show()

lower bounds aproximados:

Top 20 post ordenador por el limite inferior:



**Respuesta:**

`Segun los resultados, los post nostalgicos son más probables de tener votos positivos, por el contrario, los clickbaits ya no son buen visto en reddit `
`La solución es efectiva, pero perfectible, quizas un analisis del texto para ver si ciertas palabras claves aumentan la probabilidad de tener más votos positivos`

<p align="center">
  <img src="https://media1.tenor.com/images/fb5bf7cc5a4acb91b4177672886a88ba/tenor.gif?itemid=5591338">
</p>